## Dog Breed Classification

In this project we will use traditional CNN, CNN with data augmentation and finally transfer Learning by VGG16 model with weights pre-trained on Imagenet to solve the dog breed classification problem

### Load Dataset Files

In [0]:
from google.colab import drive

In [27]:
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


Now, upload the given dataset file shared with you in your google drive and give its path for the below given `project_path` variable. For example, a path is given below according to the file path in our google drive. You need to change this to match the path of yours.

In [0]:
project_path = "/content/drive/My Drive/Machine learning And AI/Data for Dogg breed/"

Run the below code to extract all the images in the train.zip files given in the dataset. We are going to use these images as train and validation sets and their labels in further steps.

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'train.zip', 'r') as z:
  z.extractall()

Repeat the same step for test.zip

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'test.zip', 'r') as z:
  z.extractall()

Repeat the same step for sample_submission.csv.zip

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'sample_submission.csv.zip', 'r') as z:
  z.extractall()

Repeat the same step for labels.csv.zip

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'labels.csv.zip', 'r') as z:
  z.extractall()

After this process, we will have 4 files - Train folder, test folder and labels.csv and sample_submission.csv as part of your google drive

### Read labels.csv file using pandas

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [0]:
labels = pd.read_csv("labels.csv")

In [35]:
labels.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [36]:
labels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10222 entries, 0 to 10221
Data columns (total 2 columns):
id       10222 non-null object
breed    10222 non-null object
dtypes: object(2)
memory usage: 159.8+ KB


### Print the count of each category of Dogs given in the dataset



In [37]:
labels["breed"].value_counts().head()

scottish_deerhound      126
maltese_dog             117
afghan_hound            116
entlebucher             115
bernese_mountain_dog    114
Name: breed, dtype: int64

### Get one-hot encodings of labels

In [0]:
labels.breed = pd.Categorical(labels.breed)


In [0]:
labels['breed_category'] = labels.breed.cat.codes

In [40]:
labels.head(5)

,id,breed,breed_category
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull,19
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo,37
2,001cdf01b096e06d78e9e5112d419397,pekinese,85
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick,15
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever,49


In [0]:
labels_encode = tf.keras.utils.to_categorical(labels['breed_category'],num_classes=120,dtype='float32')
labels = labels.drop(["breed_category"],axis = 1)

In [42]:
labels_encode[1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0.], dtype=float32)

## Preparing training dataset
1. Write a code which reads each and every id from labels.csv file and loads the corresponding image (in RGB - 128, 128, 3) from the train folder. <br>
2. Create 2 variables <br> 
     a.  x_train - Should have all the images of the dogs from train folder <br>
     b.  y_train - Corresponding label of the dog <br>
<u>Note:</u> The id of the dog images and its corresponding labels are available in labels.csv file   
<u>Hint:</u> Watch the video shared on "Preparing the training dataset" if you face issue on creating the training dataset

In [0]:
img_rows = 128
img_cols = 128
num_channel = 3

In [44]:
from tqdm import tqdm
import cv2
x_train = []
y_train = []

for f,img in tqdm(labels.values):
  train_img = cv2.imread('./train/{}.jpg'.format(f),1)
  train_img_resize = cv2.resize(train_img,(img_rows,img_cols))
  x_train.append(train_img_resize)
  y_train.append(img)

100%|██████████| 10222/10222 [00:25<00:00, 394.85it/s]


Normalize the training data and convert into 4 dimensions so that it can be used as an input to conv layers in the model

In [0]:
x_train = np.asarray(x_train)

In [0]:
y_train = np.asarray(y_train)

In [47]:
y_train.shape

(10222,)

In [0]:
x_train = x_train/255

### Split the training and validation data from `x_train_data` and `y_train_data` obtained from above step

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_train, X_validation, Y_train, Y_validation = train_test_split(x_train, labels_encode, test_size=.20, random_state=7)

In [0]:
del y_train

### Loading the test data
Read the id column from the samples_submission.csv and store it in test_img

In [0]:
test_img = pd.read_csv("sample_submission.csv")

In [0]:
test_img=test_img["id"]

In [0]:
#test_img.head(2)

Run the below code to load the test image files in x_test_feature

In [0]:
x_test_feature = []
i = 0 # initialisation
for f in tqdm(test_img.values): # f for format ,jpg
    img = cv2.imread('./test/{}.jpg'.format(f), 0)
    img_resize = cv2.resize(img, (img_rows, img_cols)) 
    x_test_feature.append(img_resize)

100%|██████████| 10357/10357 [00:14<00:00, 692.45it/s]


Normalize the test data and convert it into 4 dimensions

In [0]:
x_test_feature = np.asanyarray(x_test_feature)

In [0]:
x_test_feature = x_test_feature/255

In [0]:
#Code to delete unused variables


### Build a basic conv neural network with 2 conv layers (kernel sizes - 5 and 3) add layers as mentioned below for classification.

1. Add a Dense layer with 256 neurons with `relu` activation

2. Add a Dense layer with 120 neurons as final layer (as there are 120 classes in the given dataset) with `softmax` activation for classifiaction. 

In [0]:
from keras import Sequential
from keras.layers import Dense,Activation,Flatten
from keras.layers import Conv2D, MaxPooling2D, Dropout

In [33]:
cnn = Sequential()
cnn.add(Conv2D(32, (5, 5), activation='relu', input_shape=(128, 128, 3)))
cnn.add(Conv2D(32, (3, 3), activation='relu'))

cnn.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
cnn.add(Dense(256, activation='relu'))

cnn.add(Dense(120, activation='softmax'))

W0619 18:14:19.660928 140387768059776 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0619 18:14:19.721809 140387768059776 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0619 18:14:19.750306 140387768059776 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [0]:
cnn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 124, 124, 32)      2432      
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 122, 122, 32)      9248      
_________________________________________________________________
flatten_5 (Flatten)          (None, 476288)            0         
_________________________________________________________________
dense_11 (Dense)             (None, 256)               121929984 
_________________________________________________________________
dense_12 (Dense)             (None, 120)               30840     
Total params: 121,972,504
Trainable params: 121,972,504
Non-trainable params: 0
_________________________________________________________________


### Use batch_size = 128 and epochs = 10 and execute the model

In [38]:
cnn.compile(loss='categorical_crossentropy', optimizer="sgd", metrics=['accuracy'])

W0619 18:18:28.360494 140387768059776 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0619 18:18:28.389461 140387768059776 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



In [0]:
cnn.fit(X_train,Y_train,          
          validation_data=(X_validation,Y_validation),
          epochs=10,
          batch_size=128)

Train on 8177 samples, validate on 2045 samples
Epoch 1/10
8177/8177 [==============================] - 15s 2ms/step - loss: 4.7802 - acc: 0.0119 - val_loss: 4.7746 - val_acc: 0.0103
Epoch 2/10
8177/8177 [==============================] - 13s 2ms/step - loss: 4.7377 - acc: 0.0164 - val_loss: 4.7449 - val_acc: 0.0196
Epoch 3/10
8177/8177 [==============================] - 13s 2ms/step - loss: 4.6923 - acc: 0.0276 - val_loss: 4.7063 - val_acc: 0.0196
Epoch 4/10
8177/8177 [==============================] - 13s 2ms/step - loss: 4.6413 - acc: 0.0292 - val_loss: 4.6779 - val_acc: 0.0259
Epoch 5/10
8177/8177 [==============================] - 13s 2ms/step - loss: 4.5752 - acc: 0.0395 - val_loss: 4.6504 - val_acc: 0.0259
Epoch 6/10
8177/8177 [==============================] - 13s 2ms/step - loss: 4.4924 - acc: 0.0477 - val_loss: 4.6068 - val_acc: 0.0328
Epoch 7/10
8177/8177 [==============================] - 13s 2ms/step - loss: 4.3984 - acc: 0.0574 - val_loss: 4.5798 - val_acc: 0.0357
Epoch 8

#The model accuracy is very poor !!!!

### Use Data Augmentation in the above model to see if the accuracy improves


In [56]:
from keras.preprocessing.image import ImageDataGenerator , load_img ,img_to_array

Using TensorFlow backend.


In [0]:
# this will do preprocessing and realtime data augmentation
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    rotation_range=25,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=False,  # randomly flip images
    vertical_flip=False)  # randomly flip images

### Using the above objects, create the image generators with variable names `train_generator` and `val_generator`

You need to use train_datagen.flow() and val_datagen.flow()

In [0]:
train_generator = datagen.flow(X_train, Y_train, batch_size=32)
val_generator = datagen.flow(X_validation, Y_validation, batch_size=32)

In [62]:
tf.keras.backend.clear_session()

from keras.models import Sequential

from keras.layers import Conv2D, MaxPooling2D, Activation, Flatten, Dense, Dropout

#Initialize model, reshape & normalize data
model = tf.keras.models.Sequential()

#Add first convolutional layer
model.add(tf.keras.layers.Conv2D(32, #Number of filters 
                                 kernel_size=(5,5), #Size of the filter
                                 activation='relu'
                                , input_shape=(128,128,3)))



#Add second  convolutional layer
model.add(tf.keras.layers.Conv2D(32, #Number of filters 
                                 kernel_size=(3,3), #Size of the filter
                                 activation='relu'
                                , input_shape=(128,128,3)))

model.add(tf.keras.layers.Dropout(0.25))

#Flatten the output
model.add(tf.keras.layers.Flatten())

#Dense layer
model.add(tf.keras.layers.Dense(256, activation='relu'))

#Dense layer
model.add(tf.keras.layers.Dense(120, activation='softmax'))


model.compile(optimizer='adam', 
              loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 124, 124, 32)      2432      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 122, 122, 32)      9248      
_________________________________________________________________
dropout (Dropout)            (None, 122, 122, 32)      0         
_________________________________________________________________
flatten (Flatten)            (None, 476288)            0         
_________________________________________________________________
dense (Dense)                (None, 256)               121929984 
_________________________________________________________________
dense_1 (Dense)              (None, 120)               30840     
Total params: 121,972,504
Trainable params: 121,972,504
Non-trainable params: 0
__________________________________________

### Fit the model using fit_generator() using `train_generator` and `val_generator` from the above step with 10 epochs

In [63]:
# Set up the generator
#Train the model
history = model.fit_generator(train_generator, 
                             validation_data=val_generator , use_multiprocessing=True,
                     steps_per_epoch=len(X_train) / 32, epochs=10)

Epoch 1/10
256/255 [==============================] - 45s 178ms/step - loss: 5.2474 - acc: 0.0113 - val_loss: 4.7812 - val_acc: 0.0137
Epoch 2/10
256/255 [==============================] - 41s 160ms/step - loss: 4.7355 - acc: 0.0191 - val_loss: 4.7198 - val_acc: 0.0093
Epoch 3/10
256/255 [==============================] - 41s 162ms/step - loss: 4.6533 - acc: 0.0227 - val_loss: 4.7180 - val_acc: 0.0186
Epoch 4/10
256/255 [==============================] - 42s 163ms/step - loss: 4.5911 - acc: 0.0289 - val_loss: 4.6310 - val_acc: 0.0186
Epoch 5/10
256/255 [==============================] - 41s 161ms/step - loss: 4.5367 - acc: 0.0352 - val_loss: 4.5884 - val_acc: 0.0244
Epoch 6/10
256/255 [==============================] - 41s 161ms/step - loss: 4.4916 - acc: 0.0339 - val_loss: 4.5589 - val_acc: 0.0289
Epoch 7/10
256/255 [==============================] - 42s 163ms/step - loss: 4.4833 - acc: 0.0373 - val_loss: 4.5867 - val_acc: 0.0196
Epoch 8/10
256/255 [==============================] - 4

# Model accuracy is still poor!!!

### Lets use Transfer Learning

Download the vgg wieght file from here : https://github.com/MinerKasch/applied_deep_learning/blob/master/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5

Use the below code to load VGG16 weights trained on ImageNet

In [71]:
from keras.applications.vgg16 import VGG16, preprocess_input
# Instantiate the model with the pre-trained weights (no top)
base_model= VGG16(weights=(project_path+'vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'),
                 include_top=False, pooling='avg')

W0619 20:30:16.389588 139806548502400 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0619 20:30:16.392287 139806548502400 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0619 20:30:16.405667 139806548502400 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0619 20:30:16.435489 139806548502400 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0619 20:30:17.992282 139806548502400 deprecation_wrapp

Print the summary of the base_model

In [72]:
base_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

### Add the following classification layers to the imported VGG Model <br>
1. Flatten Layer
2. Dense layer with 1024 neurons with activation as Relu
3. Dense layer with 256 neurons with activation as Relu
4. Dense layer with 120 neurons with activation as Softmax

### Make all the layers in the base_model (VGG16) to be non-trainable

In [0]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout 
from keras.layers.normalization import BatchNormalization


model_transfer = Sequential()
model_transfer.add(Dense(1024,input_dim=512, activation='relu'))
model_transfer.add(BatchNormalization())
model_transfer.add(Dropout(0.80))
model_transfer.add(Dense(256, activation='relu'))
model_transfer.add(BatchNormalization())
model_transfer.add(Dropout(0.20))
model_transfer.add(Dense(256, activation='relu'))
model_transfer.add(BatchNormalization())
model_transfer.add(Dropout(0.40))

model_transfer.add(Dense(120,activation='softmax'))
model_transfer.compile(optimizer='adam', 
              loss='categorical_crossentropy', metrics=['accuracy'])

In [106]:
#Freezing layers in the model which don't have 'dense' in their name
for layer in base_model.layers:
  if('dense' not in layer.name): #prefix detection to freeze layers which does not have dense
    #Freezing a layer
    layer.trainable = False

#Module to print colourful statements
from termcolor import colored

#Check which layers have been frozen 
for layer in base_model.layers:
  print (colored(layer.name, 'blue'))
  print (colored(layer.trainable, 'red'))

input_1
False
block1_conv1
False
block1_conv2
False
block1_pool
False
block2_conv1
False
block2_conv2
False
block2_pool
False
block3_conv1
False
block3_conv2
False
block3_conv3
False
block3_pool
False
block4_conv1
False
block4_conv2
False
block4_conv3
False
block4_pool
False
block5_conv1
False
block5_conv2
False
block5_conv3
False
block5_pool
False
global_average_pooling2d_1
False


### Fit and compile the model with batch_size = 128 and epochs = 10 and execute the model

Try to get training and validation accuracy to be more than 90%

In [0]:
x_train_5 = base_model.predict(X_train)

X_valid_5 = base_model.predict(X_validation)

In [108]:
output_transfer = model_transfer.fit(x_train_5, Y_train, 
                                      validation_data=(X_valid_5, Y_validation),
                                      batch_size=128, epochs=100, verbose=1)

Train on 8177 samples, validate on 2045 samples
Epoch 1/100
8177/8177 [==============================] - 3s 324us/step - loss: 5.5075 - acc: 0.0125 - val_loss: 4.5129 - val_acc: 0.0435
Epoch 2/100
8177/8177 [==============================] - 1s 75us/step - loss: 4.8710 - acc: 0.0325 - val_loss: 4.1149 - val_acc: 0.0949
Epoch 3/100
8177/8177 [==============================] - 1s 72us/step - loss: 4.4838 - acc: 0.0558 - val_loss: 3.8726 - val_acc: 0.1134
Epoch 4/100
8177/8177 [==============================] - 1s 75us/step - loss: 4.1921 - acc: 0.0762 - val_loss: 3.7500 - val_acc: 0.1320
Epoch 5/100
8177/8177 [==============================] - 1s 74us/step - loss: 3.9504 - acc: 0.1016 - val_loss: 3.5935 - val_acc: 0.1472
Epoch 6/100
8177/8177 [==============================] - 1s 73us/step - loss: 3.8224 - acc: 0.1198 - val_loss: 3.4876 - val_acc: 0.1623
Epoch 7/100
8177/8177 [==============================] - 1s 75us/step - loss: 3.6916 - acc: 0.1412 - val_loss: 3.3886 - val_acc: 0.1927